In [1]:
import requests as rq
import time
import pandas as pd
import json
import math

In [2]:
headers = {'User-Agent': "vkrishnaswamy489@gmail.com"}

In [3]:
tickers_cik = rq.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

In [4]:
tickers_cik =     pd.json_normalize(pd.json_normalize(tickers_cik.json(),\
max_level=0).values[0])
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)
tickers_cik.set_index("ticker",inplace=True)

In [5]:
tickers_cik.sort_index()

cik_str                             title
ticker                                              
A       0001090872        AGILENT TECHNOLOGIES, INC.
AA      0001675149                        Alcoa Corp
AAAU    0001708646   Goldman Sachs Physical Gold ETF
AAC     0001829432             Ares Acquisition Corp
AAC-UN  0001829432             Ares Acquisition Corp
...            ...                               ...
ZYME    0001937653                    Zymeworks Inc.
ZYNE    0001621443     Zynerba Pharmaceuticals, Inc.
ZYRX    0001957401      GLOBAL EARNINGS CAPITAL LTD.
ZYXI    0000846475                         ZYNEX INC
ZZLL    0001365357  ZZLL INFORMATION TECHNOLOGY, INC

[11049 rows x 2 columns]

In [6]:
companies = ['NVDA', 'TSLA', 'META', 'BABA', 'CRM', 'AMD', 'INTC', 'PYPL', 'ATVI', 'EA', 'TTD',
    'MTCH', 'ZG', 'YELP', 'CSCO', 'ADBE', 'AGYS', 'MCHP', 'PD', 'DOCU', 'BOX', 'PRO',
    'VRNS', 'AYX', 'SMAR', 'AVGO', 'ACN', 'TXN', 'ORCL', 'MSFT', 'DELL', 'AMZN',
    'GOOG', 'AAPL', 'VZ', 'T', 'CMCSA', 'TMUS', 'RTX', 'SNX', 'IBM', 'HPQ', 'CHTR', 'QCOM', 'BBY', 'FISV', 
    'CTSH', 'DISH', 'ADI', 'ADP']
companies=['NVDA']

In [7]:
comp_cik = []

for company in companies:
    try:
        comp_cik.append(tickers_cik.at[company, 'cik_str'])
    except KeyError:
        print(company)

In [8]:
params = ['Assets', 'Liabilities', 'StockholdersEquity', 'InventoryNet',
          'CostOfGoodsAndServicesSold', 'GrossProfit', 'WeightedAverageNumberOfSharesOutstandingBasic',
          'CommonStockDividendsPerShareCashPaid']
params = ['Assets']

In [9]:
others = ['Name', 'Date']
df = pd.DataFrame(columns=(others+params))

In [10]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={'Name':companies[ind]}
    for num, param in enumerate(params):
        new_row = {}
        try:
            timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
        except KeyError:
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
            except KeyError:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                
        timeseries["filed"] = pd.to_datetime(timeseries["filed"])
        try:
            timeseries["start"] = pd.to_datetime(timeseries["start"])
            timeseries["end"] = pd.to_datetime(timeseries["end"])
        except Exception:
            pass

        timeseries['frame'] = timeseries['frame'].fillna(0)

        #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
        for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                # print(timeseries.tail(10))
                timeseries = timeseries.drop(j)
                timeseries = timeseries.reset_index(drop=True)
                # print(timeseries.tail(10))
        
        for i in range(0,num_per,1):
            new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
            new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
            df.loc[length+i] = new_row
        print('Current param:', param, sep=' ')
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Number of companies: 1


Name        Date        Assets
0   NaN  2022-10-30  4.048800e+10
1   NaN  2022-07-31  4.347600e+10
2   NaN  2022-05-01  4.521200e+10
3   NaN  2021-10-31  4.063200e+10
4   NaN  2021-08-01  3.865000e+10
5   NaN  2021-05-02  3.079600e+10
6   NaN  2020-10-25  2.688100e+10
7   NaN  2020-07-26  2.518000e+10
8   NaN  2020-04-26  2.325400e+10
9   NaN  2019-10-27  1.581000e+10
10  NaN  2019-07-28  1.477500e+10
11  NaN  2019-04-28  1.402100e+10
12  NaN  2018-10-28  1.365700e+10
13  NaN  2018-07-29  1.288200e+10
14  NaN  2018-04-29  1.146000e+10
15  NaN  2018-01-28  1.124100e+10
16  NaN  2018-01-28  1.124100e+10
17  NaN  2017-10-29  9.830000e+09
18  NaN  2017-07-30  9.402000e+09
19  NaN  2017-04-30  9.410000e+09

In [11]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={}
    for num, param in enumerate(params):
        new_row = {'Name':companies[ind]}
        try:
            timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
        except KeyError:
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
            except KeyError:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                
        timeseries["filed"] = pd.to_datetime(timeseries["filed"])
        try:
            timeseries["start"] = pd.to_datetime(timeseries["start"])
            timeseries["end"] = pd.to_datetime(timeseries["end"])
        except Exception:
            pass

        timeseries['frame'] = timeseries['frame'].fillna(0)

        #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
        for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                # print(timeseries.tail(10))
                timeseries = timeseries.drop(j)
                timeseries = timeseries.reset_index(drop=True)
                # print(timeseries.tail(10))
        
        for i in range(0,num_per,1):
            new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
            new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
            df.loc[length+i] = new_row
        print('Current param:', param, sep=' ')
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Number of companies: 1


Name        Date        Assets
0   NVDA  2022-10-30  4.048800e+10
1   NVDA  2022-07-31  4.347600e+10
2   NVDA  2022-05-01  4.521200e+10
3   NVDA  2021-10-31  4.063200e+10
4   NVDA  2021-08-01  3.865000e+10
5   NVDA  2021-05-02  3.079600e+10
6   NVDA  2020-10-25  2.688100e+10
7   NVDA  2020-07-26  2.518000e+10
8   NVDA  2020-04-26  2.325400e+10
9   NVDA  2019-10-27  1.581000e+10
10  NVDA  2019-07-28  1.477500e+10
11  NVDA  2019-04-28  1.402100e+10
12  NVDA  2018-10-28  1.365700e+10
13  NVDA  2018-07-29  1.288200e+10
14  NVDA  2018-04-29  1.146000e+10
15  NVDA  2018-01-28  1.124100e+10
16  NVDA  2018-01-28  1.124100e+10
17  NVDA  2017-10-29  9.830000e+09
18  NVDA  2017-07-30  9.402000e+09
19  NVDA  2017-04-30  9.410000e+09

In [12]:
params = ['Assets', 'Liabilities', 'StockholdersEquity', 'InventoryNet',
          'CostOfGoodsAndServicesSold', 'GrossProfit', 'WeightedAverageNumberOfSharesOutstandingBasic',
          'CommonStockDividendsPerShareCashPaid']
# params = ['Assets']

In [13]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={}
    for num, param in enumerate(params):
        new_row = {'Name':companies[ind]}
        try:
            timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
        except KeyError:
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
            except KeyError:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                
        timeseries["filed"] = pd.to_datetime(timeseries["filed"])
        try:
            timeseries["start"] = pd.to_datetime(timeseries["start"])
            timeseries["end"] = pd.to_datetime(timeseries["end"])
        except Exception:
            pass

        timeseries['frame'] = timeseries['frame'].fillna(0)

        #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
        for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                # print(timeseries.tail(10))
                timeseries = timeseries.drop(j)
                timeseries = timeseries.reset_index(drop=True)
                # print(timeseries.tail(10))
        
        for i in range(0,num_per,1):
            new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
            new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
            df.loc[length+i] = new_row
        print('Current param:', param, sep=' ')
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet


KeyError: -1

In [14]:
param

'CostOfGoodsAndServicesSold'

In [15]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={}
    for num, param in enumerate(params):
        print('Current param:', param, sep=' ')
        try:
            new_row = {'Name':companies[ind]}
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
            except KeyError:
                try:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
                except KeyError:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                    
            timeseries["filed"] = pd.to_datetime(timeseries["filed"])
            try:
                timeseries["start"] = pd.to_datetime(timeseries["start"])
                timeseries["end"] = pd.to_datetime(timeseries["end"])
            except Exception:
                pass

            timeseries['frame'] = timeseries['frame'].fillna(0)

            #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
                if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                    # print(timeseries.tail(10))
                    timeseries = timeseries.drop(j)
                    timeseries = timeseries.reset_index(drop=True)
                    # print(timeseries.tail(10))
            
            for i in range(0,num_per,1):
                new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
                new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
                df.loc[length+i] = new_row
        except Exception:
            pass
        
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet
Current param: CostOfGoodsAndServicesSold
Current param: GrossProfit
Current param: WeightedAverageNumberOfSharesOutstandingBasic
Current param: CommonStockDividendsPerShareCashPaid
Number of companies: 1


Name                 Date  Assets  Liabilities  StockholdersEquity   
0   NVDA  2022-10-30 00:00:00     NaN          NaN                 NaN  \
1   NVDA  2022-07-31 00:00:00     NaN          NaN                 NaN   
2   NVDA  2022-05-01 00:00:00     NaN          NaN                 NaN   
3   NVDA  2021-10-31 00:00:00     NaN          NaN                 NaN   
4   NVDA  2021-08-01 00:00:00     NaN          NaN                 NaN   
5   NVDA  2021-05-02 00:00:00     NaN          NaN                 NaN   
6   NVDA  2020-10-25 00:00:00     NaN          NaN                 NaN   
7   NVDA  2020-07-26 00:00:00     NaN          NaN                 NaN   
8   NVDA  2020-04-26 00:00:00     NaN          NaN                 NaN   
9   NVDA  2019-10-27 00:00:00     NaN          NaN                 NaN   
10  NVDA  2019-07-28 00:00:00     NaN          NaN                 NaN   
11  NVDA  2019-07-28 00:00:00     NaN          NaN                 NaN   
12  NVDA  2019-07-28 00:00:00     NaN          NaN                 NaN   
13  NVDA  2019-07-28 00:00:00     NaN          NaN                 NaN   
14  NVDA  2019-04-28 00:00:00     NaN          NaN                 NaN   
15  NVDA  2019-04-28 00:00:00     NaN          NaN                 NaN   
16  NVDA  2019-01-27 00:00:00     NaN          NaN                 NaN   
17  NVDA  2019-01-27 00:00:00     NaN          NaN                 NaN   
18  NVDA  2019-01-27 00:00:00     NaN          NaN                 NaN   
19  NVDA  2018-10-28 00:00:00     NaN          NaN                 NaN   

    InventoryNet  CostOfGoodsAndServicesSold  GrossProfit   
0            NaN                         NaN          NaN  \
1            NaN                         NaN          NaN   
2            NaN                         NaN          NaN   
3            NaN                         NaN          NaN   
4            NaN                         NaN          NaN   
5            NaN                         NaN          NaN   
6            NaN                         NaN          NaN   
7            NaN                         NaN          NaN   
8            NaN                         NaN          NaN   
9            NaN                         NaN          NaN   
10           NaN                         NaN          NaN   
11           NaN                         NaN          NaN   
12           NaN                         NaN          NaN   
13           NaN                         NaN          NaN   
14           NaN                         NaN          NaN   
15           NaN                         NaN          NaN   
16           NaN                         NaN          NaN   
17           NaN                         NaN          NaN   
18           NaN                         NaN          NaN   
19           NaN                         NaN          NaN   

    WeightedAverageNumberOfSharesOutstandingBasic   
0                                    2.483000e+09  \
1                                    2.495000e+09   
2                                    2.506000e+09   
3                                    2.499000e+09   
4                                    2.493000e+09   
5                                    2.484000e+09   
6                                    2.472000e+09   
7                                    2.464000e+09   
8                                    6.140000e+08   
9                                    6.100000e+08   
10                                   6.090000e+08   
11                                   6.090000e+08   
12                                   6.080000e+08   
13                                   6.080000e+08   
14                                   6.070000e+08   
15                                   6.070000e+08   
16                                   6.080000e+08   
17                                   6.080000e+08   
18                                   6.080000e+08   
19                                   6.090000e+08   

    CommonStockDividendsPerShareCashPaid  
0         

In [16]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={}
    for num, param in enumerate(params):
        print('Current param:', param, sep=' ')
        try:
            new_row = {'Name':companies[ind]}
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
            except KeyError:
                try:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
                except KeyError:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                    
            timeseries["filed"] = pd.to_datetime(timeseries["filed"])
            try:
                timeseries["start"] = pd.to_datetime(timeseries["start"])
                timeseries["end"] = pd.to_datetime(timeseries["end"])
            except Exception:
                pass

            timeseries['frame'] = timeseries['frame'].fillna(0)

            #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
                if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                    # print(timeseries.tail(10))
                    timeseries = timeseries.drop(j)
                    timeseries = timeseries.reset_index(drop=True)
                    # print(timeseries.tail(10))
            
            for i in range(0,num_per,1):
                new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
                new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
                df.at[length+i, param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
        except Exception:
            pass
        
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet
Current param: CostOfGoodsAndServicesSold
Current param: GrossProfit
Current param: WeightedAverageNumberOfSharesOutstandingBasic
Current param: CommonStockDividendsPerShareCashPaid
Number of companies: 1


Name  Date        Assets   Liabilities  StockholdersEquity  InventoryNet   
0    NaN   NaN  4.048800e+10  1.913900e+10        2.134900e+10  4.454000e+09  \
1    NaN   NaN  4.347600e+10  1.962500e+10        2.385100e+10  3.889000e+09   
2    NaN   NaN  4.521200e+10  1.889200e+10        2.632000e+10  3.163000e+09   
3    NaN   NaN  4.063200e+10  1.683400e+10        2.379800e+10  2.233000e+09   
4    NaN   NaN  3.865000e+10  1.750300e+10        2.114700e+10  2.114000e+09   
5    NaN   NaN  3.079600e+10  1.202200e+10        1.877400e+10  1.992000e+09   
6    NaN   NaN  2.688100e+10  1.154700e+10        1.533400e+10  1.495000e+09   
7    NaN   NaN  2.518000e+10  1.126600e+10        1.391400e+10  1.401000e+09   
8    NaN   NaN  2.325400e+10  1.015500e+10        1.309900e+10  1.128000e+09   
9    NaN   NaN  1.581000e+10  4.596000e+09        1.309900e+10  1.047000e+09   
10   NaN   NaN  1.477500e+10  4.439000e+09        1.220400e+10  1.204000e+09   
11   NaN   NaN  1.402100e+10  4.317000e+09        1.220400e+10  1.426000e+09   
12   NaN   NaN  1.365700e+10  4.182000e+09        1.220400e+10  1.417000e+09   
13   NaN   NaN  1.288200e+10  4.087000e+09        1.220400e+10  1.090000e+09   
14   NaN   NaN  1.146000e+10  3.743000e+09        1.220400e+10  7.970000e+08   
15   NaN   NaN  1.124100e+10  3.770000e+09        1.220400e+10  7.960000e+08   
16   NaN   NaN  1.124100e+10  3.770000e+09        1.220400e+10  7.960000e+08   
17   NaN   NaN  9.830000e+09  3.477000e+09        1.220400e+10  8.570000e+08   
18   NaN   NaN  9.402000e+09  3.427000e+09        1.220400e+10  8.550000e+08   
19   NaN   NaN  9.410000e+09  3.271000e+09        1.220400e+10  8.210000e+08   

    CostOfGoodsAndServicesSold   GrossProfit   
0                          NaN  3.177000e+09  \
1                          NaN  2.915000e+09   
2                          NaN  5.431000e+09   
3                          NaN  4.631000e+09   
4                          NaN  4.215000e+09   
5                          NaN  3.629000e+09   
6                          NaN  2.960000e+09   
7                          NaN  2.275000e+09   
8                          NaN  2.004000e+09   
9                          NaN  1.916000e+09   
10                         NaN  4.753000e+09   
11                         NaN  1.541000e+09   
12                         NaN  1.541000e+09   
13                         NaN  1.541000e+09   
14                         NaN  2.837000e+09   
15                         NaN  2.837000e+09   
16                         NaN  1.296000e+09   
17                         NaN  1.296000e+09   
18                         NaN  1.296000e+09   
19                         NaN  1.207000e+09   

    WeightedAverageNumberOfSharesOutstandingBasic   
0                                    2.483000e+09  \
1                                    2.495000e+09   
2                                    2.506000e+09   
3                                    2.499000e+09   
4                                    2.493000e+09   
5                                    2.484000e+09   
6                                    2.472000e+09   
7                                    2.464000e+09   
8                                    6.140000e+08   
9                                    6.100000e+08   
10                                   6.090000e+08   
11                                   6.090000e+08   
12                                   6.080000e+08   
13                                   6.080000e+08   
14                                   6.070000e+08   
15                                   6.070000e+08   
16                                   6.080000e+08   
17                                   6.080000e+08   
18                                   6.080000e+08   
19                                   6.090000e+08   

    CommonStockDividendsPerShareCashPaid  
0                                    NaN  
1                                    NaN  
2                                    NaN  
3                           

In [17]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={}
    for num, param in enumerate(params):
        print('Current param:', param, sep=' ')
        try:
            new_row = {'Name':companies[ind]}
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
            except KeyError:
                try:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
                except KeyError:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                    
            timeseries["filed"] = pd.to_datetime(timeseries["filed"])
            try:
                timeseries["start"] = pd.to_datetime(timeseries["start"])
                timeseries["end"] = pd.to_datetime(timeseries["end"])
            except Exception:
                pass

            timeseries['frame'] = timeseries['frame'].fillna(0)

            #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
                if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                    # print(timeseries.tail(10))
                    timeseries = timeseries.drop(j)
                    timeseries = timeseries.reset_index(drop=True)
                    # print(timeseries.tail(10))
            
            for i in range(0,num_per,1):
                new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
                new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
                df.at[length+i, 'Name'] = companies[ind]
                df.at[length+i, 'Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
                df.at[length+i, param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
        except Exception:
            pass
        
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet
Current param: CostOfGoodsAndServicesSold
Current param: GrossProfit
Current param: WeightedAverageNumberOfSharesOutstandingBasic
Current param: CommonStockDividendsPerShareCashPaid
Number of companies: 1


Name                 Date        Assets   Liabilities  StockholdersEquity   
0   NVDA  2022-10-30 00:00:00  4.048800e+10  1.913900e+10        2.134900e+10  \
1   NVDA  2022-07-31 00:00:00  4.347600e+10  1.962500e+10        2.385100e+10   
2   NVDA  2022-05-01 00:00:00  4.521200e+10  1.889200e+10        2.632000e+10   
3   NVDA  2021-10-31 00:00:00  4.063200e+10  1.683400e+10        2.379800e+10   
4   NVDA  2021-08-01 00:00:00  3.865000e+10  1.750300e+10        2.114700e+10   
5   NVDA  2021-05-02 00:00:00  3.079600e+10  1.202200e+10        1.877400e+10   
6   NVDA  2020-10-25 00:00:00  2.688100e+10  1.154700e+10        1.533400e+10   
7   NVDA  2020-07-26 00:00:00  2.518000e+10  1.126600e+10        1.391400e+10   
8   NVDA  2020-04-26 00:00:00  2.325400e+10  1.015500e+10        1.309900e+10   
9   NVDA  2019-10-27 00:00:00  1.581000e+10  4.596000e+09        1.309900e+10   
10  NVDA  2019-07-28 00:00:00  1.477500e+10  4.439000e+09        1.220400e+10   
11  NVDA  2019-07-28 00:00:00  1.402100e+10  4.317000e+09        1.220400e+10   
12  NVDA  2019-07-28 00:00:00  1.365700e+10  4.182000e+09        1.220400e+10   
13  NVDA  2019-07-28 00:00:00  1.288200e+10  4.087000e+09        1.220400e+10   
14  NVDA  2019-04-28 00:00:00  1.146000e+10  3.743000e+09        1.220400e+10   
15  NVDA  2019-04-28 00:00:00  1.124100e+10  3.770000e+09        1.220400e+10   
16  NVDA  2019-01-27 00:00:00  1.124100e+10  3.770000e+09        1.220400e+10   
17  NVDA  2019-01-27 00:00:00  9.830000e+09  3.477000e+09        1.220400e+10   
18  NVDA  2019-01-27 00:00:00  9.402000e+09  3.427000e+09        1.220400e+10   
19  NVDA  2018-10-28 00:00:00  9.410000e+09  3.271000e+09        1.220400e+10   

    InventoryNet  CostOfGoodsAndServicesSold   GrossProfit   
0   4.454000e+09                         NaN  3.177000e+09  \
1   3.889000e+09                         NaN  2.915000e+09   
2   3.163000e+09                         NaN  5.431000e+09   
3   2.233000e+09                         NaN  4.631000e+09   
4   2.114000e+09                         NaN  4.215000e+09   
5   1.992000e+09                         NaN  3.629000e+09   
6   1.495000e+09                         NaN  2.960000e+09   
7   1.401000e+09                         NaN  2.275000e+09   
8   1.128000e+09                         NaN  2.004000e+09   
9   1.047000e+09                         NaN  1.916000e+09   
10  1.204000e+09                         NaN  4.753000e+09   
11  1.426000e+09                         NaN  1.541000e+09   
12  1.417000e+09                         NaN  1.541000e+09   
13  1.090000e+09                         NaN  1.541000e+09   
14  7.970000e+08                         NaN  2.837000e+09   
15  7.960000e+08                         NaN  2.837000e+09   
16  7.960000e+08                         NaN  1.296000e+09   
17  8.570000e+08                         NaN  1.296000e+09   
18  8.550000e+08                         NaN  1.296000e+09   
19  8.210000e+08                         NaN  1.207000e+09   

    WeightedAverageNumberOfSharesOutstandingBasic   
0                                    2.483000e+09  \
1                                    2.495000e+09   
2                                    2.506000e+09   
3                                    2.499000e+09   
4                                    2.493000e+09   
5                                    2.484000e+09   
6                                    2.472000e+09   
7                                    2.464000e+09   
8                                    6.140000e+08   
9                                    6.100000e+08   
10                                   6.090000e+08   
11                                   6.090000e+08   
12                                   6.080000e+08   
13                                   6.080000e+08   
14                                   6.070000e+08   
15                                   6.070000e+08   
16                                   6.080000e+08   
17                                   6.080000

In [18]:
companies = ['NVDA', 'TSLA', 'META', 'BABA', 'CRM', 'AMD', 'INTC', 'PYPL', 'ATVI', 'EA', 'TTD',
    'MTCH', 'ZG', 'YELP', 'CSCO', 'ADBE', 'AGYS', 'MCHP', 'PD', 'DOCU', 'BOX', 'PRO',
    'VRNS', 'AYX', 'SMAR', 'AVGO', 'ACN', 'TXN', 'ORCL', 'MSFT', 'DELL', 'AMZN',
    'GOOG', 'AAPL', 'VZ', 'T', 'CMCSA', 'TMUS', 'RTX', 'SNX', 'IBM', 'HPQ', 'CHTR', 'QCOM', 'BBY', 'FISV', 
    'CTSH', 'DISH', 'ADI', 'ADP']
#companies=['NVDA']

In [19]:
comp_cik = []

for company in companies:
    try:
        comp_cik.append(tickers_cik.at[company, 'cik_str'])
    except KeyError:
        print(company)

In [20]:
params = ['Assets', 'Liabilities', 'StockholdersEquity', 'InventoryNet',
          'CostOfGoodsAndServicesSold', 'GrossProfit', 'WeightedAverageNumberOfSharesOutstandingBasic',
          'CommonStockDividendsPerShareCashPaid']
# params = ['Assets']

In [21]:
others = ['Name', 'Date']
df = pd.DataFrame(columns=(others+params))

In [22]:
df = pd.DataFrame(columns=(others+params))

num_per = 20

for ind, cik in enumerate(comp_cik):

    url_string = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + cik + ".json"
    response = rq.get(url_string, headers=headers)
    time.sleep(.1)
    # print(response.json())
    # a = response.json()
    length = len(df)
    for i in range(0,num_per,1):
        df.loc[len(df)]={}
    for num, param in enumerate(params):
        print('Current param:', param, sep=' ')
        try:
            new_row = {'Name':companies[ind]}
            try:
                timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD"])
            except KeyError:
                try:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["shares"])
                except KeyError:
                    timeseries = pd.json_normalize(response.json()['facts']['us-gaap'][param]["units"]["USD/shares"])
                    
            timeseries["filed"] = pd.to_datetime(timeseries["filed"])
            try:
                timeseries["start"] = pd.to_datetime(timeseries["start"])
                timeseries["end"] = pd.to_datetime(timeseries["end"])
            except Exception:
                pass

            timeseries['frame'] = timeseries['frame'].fillna(0)

            #for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
            for j in range(len(timeseries.index)-1, len(timeseries.index)-(2*num_per), -1):
                if timeseries.at[j,'fp']=='FY' or timeseries.at[j,'form']=='10-K' or timeseries.at[j,'frame']==0:
                    # print(timeseries.tail(10))
                    timeseries = timeseries.drop(j)
                    timeseries = timeseries.reset_index(drop=True)
                    # print(timeseries.tail(10))
            
            for i in range(0,num_per,1):
                new_row['Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
                new_row[param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
                df.at[length+i, 'Name'] = companies[ind]
                df.at[length+i, 'Date'] = timeseries.at[(len(timeseries.index)-i-1), 'end']
                df.at[length+i, param] = timeseries.at[(len(timeseries.index)-i-1), 'val']
        except Exception:
            pass
        
    print('Number of companies:',(ind+1),sep=' ')
        
df

Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet
Current param: CostOfGoodsAndServicesSold
Current param: GrossProfit
Current param: WeightedAverageNumberOfSharesOutstandingBasic
Current param: CommonStockDividendsPerShareCashPaid
Number of companies: 1
Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet
Current param: CostOfGoodsAndServicesSold
Current param: GrossProfit
Current param: WeightedAverageNumberOfSharesOutstandingBasic
Current param: CommonStockDividendsPerShareCashPaid
Number of companies: 2
Current param: Assets
Current param: Liabilities
Current param: StockholdersEquity
Current param: InventoryNet
Current param: CostOfGoodsAndServicesSold
Current param: GrossProfit
Current param: WeightedAverageNumberOfSharesOutstandingBasic
Current param: CommonStockDividendsPerShareCashPaid
Number of companies: 3
Current param: Assets
Current param: Liabilities
Cur

Name                 Date        Assets   Liabilities   
0    NVDA  2022-10-30 00:00:00  4.048800e+10  1.913900e+10  \
1    NVDA  2022-07-31 00:00:00  4.347600e+10  1.962500e+10   
2    NVDA  2022-05-01 00:00:00  4.521200e+10  1.889200e+10   
3    NVDA  2021-10-31 00:00:00  4.063200e+10  1.683400e+10   
4    NVDA  2021-08-01 00:00:00  3.865000e+10  1.750300e+10   
..    ...                  ...           ...           ...   
995   ADP  2019-03-31 00:00:00  3.708870e+10  4.003630e+10   
996   ADP  2018-12-31 00:00:00  4.406550e+10  4.061430e+10   
997   ADP  2018-12-31 00:00:00  4.454550e+10  3.170770e+10   
998   ADP  2018-12-31 00:00:00  3.566110e+10  3.320300e+10   
999   ADP  2018-12-31 00:00:00  3.888690e+10  3.320300e+10   

     StockholdersEquity  InventoryNet  CostOfGoodsAndServicesSold   
0          2.134900e+10  4.454000e+09                         NaN  \
1          2.385100e+10  3.889000e+09                         NaN   
2          2.632000e+10  3.163000e+09                         NaN   
3          2.379800e+10  2.233000e+09                         NaN   
4          2.114700e+10  2.114000e+09                         NaN   
..                  ...           ...                         ...   
995        5.364400e+09           NaN                8.021600e+09   
996        5.360700e+09           NaN                8.021600e+09   
997        5.360700e+09           NaN                8.086600e+09   
998        5.399900e+09           NaN                2.092600e+09   
999        5.399900e+09           NaN                2.092600e+09   

      GrossProfit  WeightedAverageNumberOfSharesOutstandingBasic   
0    3.177000e+09                                   2.483000e+09  \
1    2.915000e+09                                   2.495000e+09   
2    5.431000e+09                                   2.506000e+09   
3    4.631000e+09                                   2.499000e+09   
4    4.215000e+09                                   2.493000e+09   
..            ...                                            ...   
995  1.368400e+09                                   4.355000e+08   
996  1.366100e+09                                   4.357000e+08   
997  1.123500e+09                                   4.357000e+08   
998  1.121200e+09                                   4.362000e+08   
999  1.051000e+09                                   4.362000e+08   

     CommonStockDividendsPerShareCashPaid  
0                                     NaN  
1                                     NaN  
2                                     NaN  
3                                     NaN  
4                                     NaN  
..                                    ...  
995                                   NaN  
996                                   NaN  
997                                   NaN  
998                                   NaN  
999                                   NaN  

[1000 rows x 10 columns]

In [23]:
import openpyxl

In [24]:
df.to_csv("trial.csv",mode='a')

In [25]:
df.to_csv("trial.csv",mode='w')